In [8]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
from datasets import load_dataset

import os

In [2]:
model_repo = "yhavinga/gpt-neo-125m-dutch"

config = AutoConfig.from_pretrained(model_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo, config=config)
model = AutoModelForCausalLM.from_pretrained(model_repo, config=config)

if config.pad_token_id is None:
    tokenizer.pad_token_id = config.eos_token_id
else:
    tokenizer.pad_token_id = config.pad_token_id

In [3]:
layers_to_freeze = 10

for name, param in model.transformer.named_parameters():
    print(name)

    if layers_to_freeze:
        if "h." in name:
            # Extract the layer number from the parameter name
            layer_num = int(name.split(".")[1])
        else:
            layer_num = None
            to_freeze = layer_num and layer_num < layers_to_freeze
    else:
        # if no layers are being frozen, set to_freeze to false for all layers
        to_freeze = False

    # Freeze the current parameter if it belongs to the embedding layer
    # or if to_freeze is true for this layer
    if name == "wte.weight" or to_freeze:
        param.requires_grad = False

wte.weight
wpe.weight
h.0.ln_1.weight
h.0.ln_1.bias
h.0.attn.attention.k_proj.weight
h.0.attn.attention.v_proj.weight
h.0.attn.attention.q_proj.weight
h.0.attn.attention.out_proj.weight
h.0.attn.attention.out_proj.bias
h.0.ln_2.weight
h.0.ln_2.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_proj.weight
h.0.mlp.c_proj.bias
h.1.ln_1.weight
h.1.ln_1.bias
h.1.attn.attention.k_proj.weight
h.1.attn.attention.v_proj.weight
h.1.attn.attention.q_proj.weight
h.1.attn.attention.out_proj.weight
h.1.attn.attention.out_proj.bias
h.1.ln_2.weight
h.1.ln_2.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_proj.weight
h.1.mlp.c_proj.bias
h.2.ln_1.weight
h.2.ln_1.bias
h.2.attn.attention.k_proj.weight
h.2.attn.attention.v_proj.weight
h.2.attn.attention.q_proj.weight
h.2.attn.attention.out_proj.weight
h.2.attn.attention.out_proj.bias
h.2.ln_2.weight
h.2.ln_2.bias
h.2.mlp.c_fc.weight
h.2.mlp.c_fc.bias
h.2.mlp.c_proj.weight
h.2.mlp.c_proj.bias
h.3.ln_1.weight
h.3.ln_1.bias
h.3.attn.attention.k_proj.w

In [4]:
trainings_data = load_dataset('csv', data_files='./trainings_data/short_stories_1117.csv')

trainings_data

Found cached dataset csv (/home2/response/.cache/huggingface/datasets/csv/default-40f43ffa42a8b736/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 1117
    })
})

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["0"], padding="max_length", truncation=True, max_length=1024)

tokenized_datasets = trainings_data.map(tokenize_function, batched=True)

tokenized_datasets["train"] = tokenized_datasets["train"].add_column("labels", tokenized_datasets["train"]["input_ids"])

tokenized_datasets

Loading cached processed dataset at /home2/response/.cache/huggingface/datasets/csv/default-40f43ffa42a8b736/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-32d8f24da1524aba.arrow


DatasetDict({
    train: Dataset({
        features: ['0', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1117
    })
})

In [6]:
training_args = TrainingArguments(
    output_dir="./output_transformers", 
    # dataloader_num_workers=os.cpu_count(),
    per_device_train_batch_size=4,
    label_names=["input_ids"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"]
)

trainer.train()

/home2/response/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home2/response/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=420, training_loss=2.4462367466517856, metrics={'train_runtime': 640.4643, 'train_samples_per_second': 5.232, 'train_steps_per_second': 0.656, 'total_flos': 1750609966399488.0, 'train_loss': 2.4462367466517856, 'epoch': 3.0})

In [19]:
generator = pipeline(
  'text-generation',
  model,
  tokenizer=tokenizer,
  config=config,
  device='cuda:0'
)

print(
    generator(
      "Leuk je te ontmoeten, waar ben je geweest?\n"
      "Ik zou je ongelofelijke dingen kunnen laten zien\n"
      "Magie, gekte, hemel, zonde\n"
      "Zag je daar en ik dacht\n"
      "“Oh mijn God, kijk naar dat gezicht\n"
      "Jij ziet eruit als mijn volgende fout\n"
      "Liefde is een spel, wil je spelen?”, zei ik\n",
      min_length=100,
      max_length=200,
      temperature=0.9,
      top_k=50,
      top_p=0.95,
      repetition_penalty=1.2
  )[0]["generated_text"]
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Leuk je te ontmoeten, waar ben je geweest?
Ik zou je ongelofelijke dingen kunnen laten zien
Magie, gekte, hemel, zonde
Zag je daar en ik dacht
“Oh mijn God, kijk naar dat gezicht
Jij ziet eruit als mijn volgende fout
Liefde is een spel, wil je spelen?”, zei ik
ik keek om me heen maar zag niets meer
de wereld was nog niet klaar voor jou
en jij had het al gezien
je bent hier gekomen omdat je zo van de ene op de andere dag in een grot terecht kwam
het was alsof er iets mis ging met jouw ogen die dicht tegen elkaar aan zaten
dat deed pijn
maar wat gebeurde er dan
wat doe je hier
jij kijkt mij aan
ik zie geen enkel teken van leven
een klein beetje licht of donker kan alles zijn
als ik jou nu vraag hoe heet je nou eigenlijk
hoe heette je ook alweer
of waarom heb je hem nooit eerder gezien
hij heeft wel
